In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib

In [2]:
train = pd.read_csv('data/train.csv',encoding = 'utf-8')
job_tag = pd.read_csv('data/job_tags.csv',encoding = 'utf-8')
job_company = pd.read_csv('data/job_companies.csv',encoding = 'utf-8')
tag = pd.read_csv('data/tags.csv',encoding = 'utf-8')
user_tag = pd.read_csv('data/user_tags.csv',encoding = 'utf-8')
test = pd.read_csv('data/test_job.csv')

In [3]:
# 중복된 행이 많아서 중복 제거
user_tag_nondeg = user_tag.drop_duplicates(['userID','tagID'])

In [4]:
# 사용된 tagID onehot
usertag = list(set(user_tag['tagID']))
jobtag = list(set(job_tag['tagID']))
total = list(set(usertag + jobtag))

onehot = OneHotEncoder()
onehot.fit(np.array(total).reshape(-1,1))

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [5]:
# tagID 이용해 userID 벡터화

tagID_onehot = onehot.transform(np.array(user_tag_nondeg['tagID']).reshape(-1,1)).toarray()

user2tag = {k:[np.zeros(419)] for k in set(user_tag['userID'])}
for i in range(len(user_tag_nondeg)):
    user2tag[user_tag_nondeg['userID'].iloc[i]]+=tagID_onehot[i]

In [6]:
# jobID를 벡터화 (tagID, companyID, companySize)
job2tagID_onehot = onehot.transform(np.array(job_tag['tagID']).reshape(-1,1)).toarray()

job2tag = {k:[np.zeros(419)] for k in set(job_tag['jobID'])}
for i in range(len(job_tag)):
    job2tag[job_tag['jobID'].iloc[i]]+= job2tagID_onehot[i]

In [7]:
# train dataset 준비
train['Xvec'] = [np.array(list(user2tag[i][0])+list(job2tag[j][0])) for i,j in zip(train['userID'],train['jobID'])]

before_pca = np.array([x.reshape(-1,1) for x in train['Xvec']])

A = before_pca
A = np.asmatrix(A.T) * np.asmatrix(A)
U, S, V = np.linalg.svd(A) 
eigvals = S**2 / np.sum(S**2)  # NOTE (@amoeba): These are not PCA eigenvalues. 
                               # This question is about SVD.

fig = plt.figure(figsize=(8,5))
sing_vals = np.arange(838) + 1
plt.plot(sing_vals, eigvals, 'ro-', linewidth=2)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue')
#I don't like the default legend so I typically make mine like below, e.g.
#with smaller fonts and a bit transparent so I do not cover up data, and make
#it moveable by the viewer in case upper-right is a bad place for it 
leg = plt.legend(['Eigenvalues from SVD'], loc='best', borderpad=0.3, 
                 shadow=False, prop=matplotlib.font_manager.FontProperties(size='small'),
                 markerscale=0.4)
leg.get_frame().set_alpha(0.4)
leg.draggable(state=True)
plt.xlim(0,10)
plt.show()

# PCA로 3개의 성분으로 줄여보자
model_pca = PCA(n_components=3).fit(list(train['Xvec']))

train['pca_data'] = list(model_pca.transform(list(train['Xvec'])))

In [8]:
#trainset : validset = 5:1
train = train.sample(frac = 1, random_state = 1)
valid = train[-1000:]
train = train[:-1000]

In [9]:
valid.head()

,userID,jobID,applied,Xvec
923,65f2e6f22b24b36f9fcb364dfafeee74,1ce927f875864094e3906a4a0b5ece68,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2044,792bb50d101782d726d38be8f81f145b,a0160709701140704575d499c997b6ca,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1980,951923a83597e7db3def99b660cf0fa3,4ca82782c5372a547c104929f03fe7a9,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
249,ce3c9c905135bab43c25500d3435a7a7,310ce61c90f3a46e340ee8257bc70e93,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3911,5b34f52c8cfb9f3fa83613822ba0118c,efe937780e95574250dabe07151bdc23,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."


In [9]:
DT = tree.DecisionTreeClassifier()
DT.fit(list(train['Xvec']),list(train['applied']))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [10]:
Yhat = DT.predict(list(train['Xvec']))

In [11]:
cnt = 0
for i,j in zip(Yhat, train['applied']):
    if i==j:
        cnt+=1
print('문제수 =', len(train),
     '정답 = ', cnt,
     '정답률 = ', cnt/len(train))

문제수 = 5000 정답 =  4955 정답률 =  0.991


In [13]:
sum(Yhat)

1963

In [13]:
predict = DT.predict(list(valid['Xvec']))
cnt = 0
for i,j in zip(predict, valid['applied']):
    if i==j:
        cnt+=1
print('문제수 =', len(valid),
     '정답 = ', cnt,
     '정답률 = ', cnt/len(valid))

문제수 = 1000 정답 =  777 정답률 =  0.777


In [56]:
test = pd.read_csv('data/test_job.csv')

In [57]:
test['Xvec'] = [np.array(list(user2tag[i][0])+list(job2tag[j][0])) for i,j in zip(test['userID'],test['jobID'])]

In [59]:
model_pca = PCA(n_components=3).fit(list(test['Xvec']))

In [60]:
test['pca_data'] = list(model_pca.transform(list(test['Xvec'])))

In [61]:
predict = SVM.predict(list(test['pca_data']))

In [62]:
applied = ['applied']
applied.extend(predict)

In [63]:
len(applied)

2436

In [64]:
DFpredict = pd.DataFrame(applied)
DFpredict.to_csv(r"C:\Users\SunA\Desktop\MLjobfair\result\output17.csv",header = False, index = False)